<a href="https://colab.research.google.com/github/ZaeemAkbarKhan786/MS_ProgrammingForAI/blob/main/bert_fine_tuning_r8_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-------------------------------------------------------------

# **Name:** Zaeem Akbar Khan
# **Registration #:** FA24-RAI-019
# **Course:** Programming for AI
# **Assignment #:** 03

-------------------------------------------------------------

**Base Paper Name:** Bert-Enhanced Text Graph Neural Network for Classification

# **Selected Paper**
✔ Recent Research (2020–2024)

✔ Reputable Journal (W Category)

✔ BERT-Based Model for Text Classification

✔ Clear Description of Dataset, Problem, and Methodology


*   The problem statement (text classification).
*   Datasets (MR, SST-2, R8, Ohsumed).
*   Methodology, including graph construction, BERT-based feature extraction, and feature aggregation.

✔ Code Availability

## **Selected Dataset:**
**R8** because it offers multi-class classification and moderately long texts, allowing for meaningful evaluation and comparison.

**Link:** https://www.kaggle.com/datasets/weipengfei/ohr8r52

# **Selected Paper links:**
**PubMed:** https://pubmed.ncbi.nlm.nih.gov/34828233/

**ResearchGate:** https://www.researchgate.net/publication/356388348_Bert-Enhanced_Text_Graph_Neural_Network_for_Classification

**Semantic Scholar:** https://www.semanticscholar.org/paper/Bert-Enhanced-Text-Graph-Neural-Network-for-Yang-Cui/108eea36fc140fc0aeb2b6024925b28a7125c2ca

# **Base Paper Code:**
**Link:** https://github.com/pingpingand/BEGNN

-------------------------------------------------------------

**Step 1: Install Required Libraries**

In [ ]:
!pip install transformers torch scikit-learn numpy pandas

**Step 2: Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_scheduler

**Step 3: Load and Preprocess Dataset**

In [ ]:
df = pd.read_csv("r8-dev-stemmed.csv")
print(df.info())
print(df.columns)
df.head()

df = df.iloc[:, :2]
df.columns = ["label", "text"]

labels = df["label"].unique()
label_to_id = {label: i for i, label in enumerate(labels)}
id_to_label = {i: label for label, i in label_to_id.items()}
df["label_id"] = df["label"].map(label_to_id)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"], df["label_id"], test_size=0.2, random_state=42
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547 entries, 0 to 546
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    547 non-null    object
 1   edge    547 non-null    object
 2   intent  547 non-null    object
dtypes: object(3)
memory usage: 12.9+ KB
None
Index(['text', 'edge', 'intent'], dtype='object')


**Step 4: Tokenize Dataset**

In [ ]:
from transformers import BertTokenizer

max_length = 256
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=max_length)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

**Step 5: Convert to PyTorch Dataset**

In [ ]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, list(train_labels))
val_dataset = NewsDataset(val_encodings, list(val_labels))

**Step 6: Create DataLoader with Reduced Batch Size**

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

**Step 7: Initialize BERT Model**

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(labels))
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

**Step 8: Define Optimizer and Scheduler**

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

epochs = 3
num_training_steps = len(train_loader) * epochs
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

**Step 9: Training with Mixed Precision**

In [ ]:
from tqdm import tqdm

from torch.amp import GradScaler

scaler = GradScaler()

model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    progress_bar = tqdm(train_loader, desc="Training")
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(**batch)
            loss = outputs.loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    scheduler.step()

Epoch 1/3


Training:   0%|          | 0/55 [00:00<?, ?it/s]<ipython-input-14-3bab7d5fbdb7>:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Training: 100%|██████████| 55/55 [18:18<00:00, 19.97s/it]


Epoch 2/3


Training: 100%|██████████| 55/55 [18:14<00:00, 19.90s/it]


Epoch 3/3


Training: 100%|██████████| 55/55 [18:03<00:00, 19.69s/it]


**Step 10: Training the Model**

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

    print(f"Epoch {epoch + 1}: Loss = {total_loss / len(train_loader)}")

<ipython-input-15-d5c37471c052>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1: Loss = 6.312383365631104
Epoch 2: Loss = 6.230307431654497
Epoch 3: Loss = 6.190144634246826


**Step 11: Evaluate the Model**

In [ ]:
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average="weighted")
recall = recall_score(true_labels, predictions, average="weighted")
f1 = f1_score(true_labels, predictions, average="weighted")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.00%
Precision: 0.00
Recall: 0.00
F1 Score: 0.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Step 12: Compare with Reported Accuracy**

In [ ]:
reported_accuracy = 98.41
print(f"Our Model Accuracy: {accuracy * 100:.2f}%")
print(f"Reported Model Accuracy: {reported_accuracy}%")

Our Model Accuracy: 0.00%
Reported Model Accuracy: 98.41%
